In [1]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [2]:
import numpy as np
import torch
import sys
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

import torchvision
from torch.utils import data
from torchvision import transforms
from torchvision.datasets import MNIST

import matplotlib.pyplot as plt
import time
import zipfile
import tarfile
from PIL import Image
from torch.nn.utils.rnn import *


cuda = torch.cuda.is_available()
device = torch.device("cuda" if cuda else "cpu")
print(cuda)


True


In [3]:
with zipfile.ZipFile('/content/drive/My Drive/11-785-s20-hw4p2.zip','r') as zp:
    print('Starting to extract......')
    zp.extractall()
    print('Done!')

Starting to extract......
Done!


## Loading data


In [0]:
    speech_train = np.load('train_new.npy', allow_pickle=True, encoding='bytes')
    speech_valid = np.load('dev_new.npy', allow_pickle=True, encoding='bytes')
    speech_test = np.load('test_new.npy', allow_pickle=True, encoding='bytes')

    transcript_train = np.load('train_transcripts.npy', allow_pickle=True,encoding='bytes')
    transcript_valid = np.load('dev_transcripts.npy', allow_pickle=True,encoding='bytes')

## Adding <sos> and <eos> label to each sentence in label

In [0]:
for i in range(len(transcript_train)):
  transcript_train[i] = np.asarray(['<sos>'] + list(transcript_train[i]) + ['<eos>'])

for i in range(len(transcript_valid)):
  transcript_valid[i] = np.asarray(['<sos>'] + list(transcript_valid[i]) + ['<eos>'])


## appending sos and eos


In [6]:

print('Check train.........')
for i in range(10):
  print(transcript_train[i])

print()
print('Check valid.........')
for i in range(10):
  print(transcript_valid[i])

Check train.........
['<sos>' 'THE' 'FEMALE' 'PRODUCES' 'A' 'LITTER' 'OF' 'TWO' 'TO' 'FOUR'
 'YOUNG' 'IN' 'NOVEMBER' 'AND' 'DECEMBER' '<eos>']
['<sos>' 'NUMEROUS' 'WORKS' 'OF' 'ART' 'ARE' 'BASED' 'ON' 'THE' 'STORY'
 'OF' 'THE' 'SACRIFICE' 'OF' 'ISAAC' '<eos>']
['<sos>' 'THEIR' 'SOLUTION' 'REQUIRES' 'DEVELOPMENT' 'OF' 'THE' 'HUMAN'
 'CAPACITY' 'FOR' 'SOCIAL' 'INTEREST' '<eos>']
['<sos>' 'HIS' 'MOST' 'SIGNIFICANT' 'SCIENTIFIC' 'PUBLICATIONS' 'WERE'
 'STUDIES' 'OF' 'BIRDS' 'AND' 'ANIMALS' '<eos>']
['<sos>' 'IN' 'RECENT' 'YEARS' 'SHE' 'HAS' 'PRIMARILY' 'APPEARED' 'IN'
 'TELEVISION' 'FILMS' 'SUCH' 'AS' 'LITTLE' 'GLORIA' '<eos>']
['<sos>' 'THE' 'PROCESS' 'BY' 'WHICH' 'THE' 'LENS' 'FOCUSES' 'ON'
 'EXTERNAL' 'OBJECTS' 'IS' 'CALLED' 'ACCOMMODATION' '<eos>']
['<sos>' 'TWO' 'NARROW' 'GAUGE' 'RAILROADS' 'FROM' 'CHINA' 'ENTER' 'THE'
 'CITY' 'FROM' 'THE' 'NORTHEAST' 'AND' 'NORTHWEST' '<eos>']
['<sos>' 'SOME' 'MAPS' 'USE' 'BANDS' 'OF' 'COLOR' 'TO' 'INDICATE'
 'DIFFERENT' 'INTERVALS' 'OF' 'VALUE' '<eo

In [7]:
print(len(transcript_valid))


1106


## Change labels to character based level, May expend the whole label

In [0]:
LETTER_LIST = ['<pad>', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', \
               'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', '-', "'", '.', '_', '+', ' ','<sos>','<eos>']

In [0]:
def get_label_num(word):
  result = []
  if word == '<sos>' or word == '<eos>' or word == '<pad>':
    result.append(LETTER_LIST.index(word))
    return result

  for i in word:
    result.append(LETTER_LIST.index(i))
  result.append(0)
  return result

In [0]:
def get_label(label):
  result = []
  for i in range(len(label)):
    temp = []
    for j in label[i]:
      temp += get_label_num(j)
    result.append(temp)
  return result

In [0]:
label_train = np.asarray(get_label(transcript_train))
label_valid = np.asarray(get_label(transcript_valid))

In [12]:
type(label_train)

numpy.ndarray

In [0]:
def normalize(x, m, s): return (x-m)/s
def normalize_to(train):
    m,s = train.mean(),train.std()
    return normalize(train, m, s)

In [0]:
class MyDataset(object):

    def __init__(self, X, Y):
        self.data_dict = []
        self.Y = [torch.Tensor(j) for j in Y]
        self.X = [torch.Tensor(j) for j in X]

        self.X_lens = torch.LongTensor([len(seq) for seq in X])
        self.Y_lens = torch.LongTensor([len(seq) for seq in Y])
        self.size = len(self.X)

    def __len__(self):
        return self.size

    def __getitem__(self, index):
        return self.X[index], self.X_lens[index], self.Y[index], self.Y_lens[index]
      
def my_collate(batch):
    data = [item[0] for item in batch]
    target = [item[2] for item in batch]
    data_lens = torch.LongTensor([seq[1] for seq in batch])
    target_lens = torch.LongTensor([seq[3] for seq in batch])
    for i in range(len(data)):
      data[i] = normalize_to(data[i])
    data = pad_sequence(data, batch_first=True)
    target = pad_sequence(target, batch_first=True)
    pad = torch.zeros((data.shape[0], 1, data.shape[2]))
    if data.shape[1] % 2 != 0:
      data = torch.cat((data, pad), dim = 1)
    return data, data_lens, target, target_lens

In [0]:
# train dataset
train_dataset = MyDataset(speech_train, label_train)
train_loader = data.DataLoader(train_dataset, shuffle=True, batch_size=64, collate_fn = my_collate)

# validation dataset
valid_dataset = MyDataset(speech_valid, label_valid)
valid_loader = data.DataLoader(valid_dataset, shuffle=False, batch_size=64, collate_fn = my_collate)

In [16]:
count = 0
for features, features_lens, targets, target_lens in train_loader: 
  print('features: ', features.shape)
  print('features_lens: ', features_lens)
  print('targets: ', targets.shape)
  print('target_lens: ', target_lens)
  print()
  if count == 10:
    break
  count+=1

features:  torch.Size([64, 1368, 40])
features_lens:  tensor([ 553,  584, 1094,  712,  410,  155,  904, 1094,  489,  638,  421,  583,
         585,  483,  495,  268, 1212, 1368,  720,  477,  942,  284,  517,  587,
         534,  349,  957, 1354,  506,  378,  473,  764,  721,  297,  426,  588,
         702,  477, 1101,  832,  415,  964,  752,  169,  578,  573,  957,  430,
         560, 1183,  561,  390,  174,  412,  592,  481, 1022, 1187,  795, 1058,
         937,  583,  517,  371])
targets:  torch.Size([64, 218])
target_lens:  tensor([ 65, 121, 185, 132,  57,  22, 165, 157,  82, 120,  85,  89,  82,  84,
         73,  43, 181, 180,  96,  91, 155,  39,  73,  94,  82,  64, 147, 218,
         99,  60,  75, 132, 136,  42,  73, 131, 101,  56, 216, 130,  83, 160,
        115,  24, 107,  99, 133,  82,  79, 152,  94,  64,  34,  92,  82,  76,
        168, 163, 102, 166, 183, 110,  82,  52])

features:  torch.Size([64, 1352, 40])
features_lens:  tensor([ 685,  530,  822,  782,  617,  454,  807,  

## pBLSTM

In [0]:
class pBLSTM(nn.Module):
    '''
    Pyramidal BiLSTM
    The length of utterance (speech input) can be hundereds to thousands of frames long.
    The Paper reports that a direct LSTM implementation as Encoder resulted in slow convergence,
    and inferior results even after extensive training.
    The major reason is inability of AttendAndSpell operation to extract relevant information
    from a large number of input steps.
    '''
    def __init__(self, input_dim, hidden_dim):
        super(pBLSTM, self).__init__()
        self.blstm = nn.LSTM(input_size=input_dim*2, hidden_size=hidden_dim, num_layers=1, bidirectional=True, batch_first=True)

    def forward(self, x, length):
        '''
        :param x :(T, B, N) input to the pBLSTM
        :return output: (T/2, B, N*2) encoded sequence from pyramidal Bi-LSTM 
        '''
        # change the data from B * T * H -> B * T/2 * H*2
        DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'
        pad = torch.zeros((x.shape[0], 1, x.shape[2]))
        pad = pad.to(DEVICE)
        if x.shape[1] % 2 != 0:
          x = torch.cat((x, pad), dim = 1).to(DEVICE)
        batch_size = x.shape[0]
        seq_len = x.shape[1]
        dim = x.shape[2]
        l1 = (seq_len // 2)
        # print('l1: ', l1)
        # print('temp[:l1]: ', x[:l1])
        inputs = torch.cat((x[:, :l1, :], x[:, l1:, :]), dim = 2).to(DEVICE)

        # change the each length of data from a to a / 2
        for i in range(len(length)):
          if length[i] % 2 != 0:
            length[i] = (length[i] + 1) / 2
          else:
            length[i] = length[i] / 2
        
        input_data = pack_padded_sequence(inputs, length, batch_first=True,enforce_sorted=False)
        output, _ = self.blstm(input_data)
        output, _ = pad_packed_sequence(output, batch_first=True)
        return output, length


## Listener 

In [0]:
class Encoder(nn.Module):
    '''
    Encoder takes the utterances as inputs and returns the key and value.
    Key and value are nothing but simple projections of the output from pBLSTM network.
    '''
    def __init__(self, input_dim, hidden_dim, value_size=128,key_size=128):
        super(Encoder, self).__init__()
        self.pBLSTM1 = pBLSTM(input_dim, hidden_dim)
        self.pBLSTM2 = pBLSTM(2*hidden_dim, hidden_dim)
        self.pBLSTM3 = pBLSTM(2*hidden_dim, hidden_dim)
        
        ### Add code to define the blocks of pBLSTMs! ###

        self.key_network = nn.Linear(hidden_dim*2, value_size)
        self.value_network = nn.Linear(hidden_dim*2, key_size)

    def forward(self, x, lens):
        output, input_lengths = self.pBLSTM1(x, lens)
        output, input_lengths = self.pBLSTM2(output, input_lengths)
        output, input_lengths = self.pBLSTM3(output, input_lengths)
        keys = self.key_network(output)
        value = self.value_network(output)

        return keys, value

## Attention

In [0]:
class Attention(nn.Module):
    '''
    Attention is calculated using key, value and query from Encoder and decoder.
    Below are the set of operations you need to perform for computing attention:
        energy = bmm(key, query)
        attention = softmax(energy)
        context = bmm(attention, value)
    '''
    def __init__(self):
        super(Attention, self).__init__()

    def forward(self, query, key, value, lens):
        '''
        :param query :(N, context_size) Query is the output of LSTMCell from Decoder
        :param key: (N, key_size) Key Projection from Encoder per time step
        :param value: (N, value_size) Value Projection from Encoder per time step
        :return output: Attended Context
        :return attention_mask: Attention mask that can be plotted  
        '''
        energy = torch.bmm(key.unsqueeze(2), query.unsqueeze(1))

        # Create an (batch_size, max_len) boolean mask for all padding positions
        # Make use of broadcasting: (1, max_len), (batch_size, 1) -> (batch_size, max_len)
        mask = torch.arange(key.size(1)).unsqueeze(0) >= lens.unsqueeze(1)
        
        # Set attention logits at padding positions to negative infinity.
        energy.masked_fill_(mask, -1e9)

        attention = nn.functional.softmax(energy, dim = -1)
        context = torch.bmm(attention, value.unsqueeze(2)).squeeze(2)
        return context, attention

## Decoder

In [0]:
class Decoder(nn.Module):
    '''
    As mentioned in a previous recitation, each forward call of decoder deals with just one time step, 
    thus we use LSTMCell instead of LSLTM here.
    The output from the second LSTMCell can be used as query here for attention module.
    In place of value that we get from the attention, this can be replace by context we get from the attention.
    Methods like Gumble noise and teacher forcing can also be incorporated for improving the performance.
    '''
    def __init__(self, vocab_size, hidden_dim, value_size=128, key_size=128, isAttended=False):
        super(Decoder, self).__init__()
        self.embedding = nn.Embedding(vocab_size, hidden_dim, padding_idx=0)
        self.lstm1 = nn.LSTMCell(input_size=hidden_dim + value_size, hidden_size=hidden_dim)
        self.lstm2 = nn.LSTMCell(input_size=hidden_dim, hidden_size=key_size)

        self.isAttended = isAttended
        if (isAttended == True):
            self.attention = Attention()

        self.character_prob = nn.Linear(key_size + value_size, vocab_size)

    def forward(self, key, values, text, isTrain=True):
        '''
        :param key :(T, N, key_size) Output of the Encoder Key projection layer
        :param values: (T, N, value_size) Output of the Encoder Value projection layer
        :param text: (N, text_len) Batch input of text with text_length
        :param isTrain: Train or eval mode
        :return predictions: Returns the character perdiction probability 
        '''
        batch_size = key.shape[0]
        print('key: ', key.shape)
        print('value: ', values.shape)
        print('text: ', text.shape)
        if (isTrain == True):
            max_len =  text.shape[1]
            embeddings = self.embedding(text.long())
        else:
            max_len = 250

        predictions = []
        hidden_states = [None, None]
        prediction = torch.zeros(batch_size,1).to(DEVICE)

        for i in range(max_len):
            # * Implement Gumble noise and teacher forcing techniques 
            # * When attention is True, replace values[i,:,:] with the context you get from attention.
            # * If you haven't implemented attention yet, then you may want to check the index and break 
            #   out of the loop so you do you do not get index out of range errors. 

            if (isTrain):
                char_embed = embeddings[:,i,:]
                # shape : N, 1, 35
            else:
                char_embed = self.embedding(prediction.argmax(dim=-1))

            # value[i,:,:] -> 1, N, value_size
            # print('char_embed shape: ', char_embed.shape)
            # print('values shape: ', values[:,i,:].shape)
            if i < values.shape[1]:
              inp = torch.cat([char_embed, values[:,i,:]], dim=1)
            else:
              pad = torch.zeros((char_embed.shape)).to(DEVICE)
              inp = torch.cat([char_embed, pad], dim=1)
            hidden_states[0] = self.lstm1(inp, hidden_states[0])

            inp_2 = hidden_states[0][0]
            hidden_states[1] = self.lstm2(inp_2, hidden_states[1])

            ### Compute attention from the output of the second LSTM Cell ###
            output = hidden_states[1][0]

            if self.isAttended:
              output, attention = self.attention(output, key[:,i,:], values[:,i,:], lens)

            if i < values.shape[1]:
              prediction = self.character_prob(torch.cat([output, values[:,i,:]], dim=1))
            else:
              pad = torch.zeros((output.shape)).to(DEVICE)
              inp = torch.cat([output, pad], dim=1)
            predictions.append(prediction.unsqueeze(1))

        return torch.cat(predictions, dim=1)


class Seq2Seq(nn.Module):
    '''
    We train an end-to-end sequence to sequence model comprising of Encoder and Decoder.
    This is simply a wrapper "model" for your encoder and decoder.
    '''
    def __init__(self, input_dim, vocab_size, hidden_dim, value_size=128, key_size=128, isAttended=False):
        super(Seq2Seq, self).__init__()
        self.encoder = Encoder(input_dim, hidden_dim)
        self.decoder = Decoder(vocab_size, hidden_dim)

    def forward(self, speech_input, speech_len, text_input, isTrain=True):
        key, value = self.encoder(speech_input, speech_len)
        if (isTrain == True):
            # print('text_input: ', text_input)
            predictions = self.decoder(key, value, text_input)
        else:
            predictions = self.decoder(key, value, text=None, isTrain=False)
        return predictions

## Train and test

In [0]:
DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'

def train(model, train_loader, criterion, optimizer, epoch):
    model.train()
    model.to(DEVICE)
    st = time.time()

    # 1) Iterate through your loader

        # 2) Use torch.autograd.set_detect_anomaly(True) to get notices about gradient explosion
        
            # 3) Set the inputs to the device.

            # 4) Pass your inputs, and length of speech into the model.

            # 5) Generate a mask based on the lengths of the text to create a masked loss. 
            # 5.1) Ensure the mask is on the device and is the correct shape.

            # 6) If necessary, reshape your predictions and origianl text input 
            # 6.1) Use .contiguous() if you need to. 

            # 7) Use the criterion to get the loss.

            # 8) Use the mask to calculate a masked loss. 

            # 9) Run the backward pass on the masked loss. 

            # 10) Use torch.nn.utils.clip_grad_norm(model.parameters(), 2)
            
            # 11) Take a step with your optimizer

            # 12) Normalize the masked loss

            # 13) Optionally print the training loss after every N batches
    batch_index = 1
    for features, features_lens, targets, target_lens in train_loader:
      st = time.time()
      features, features_lens, targets, target_lens = features.to(DEVICE), features_lens.to(DEVICE), targets.to(DEVICE), target_lens.to(DEVICE)
      
      predictions = model(speech_input = features, speech_len = features_lens, text_input = targets, isTrain=True)

      print('features shape: ', features.shape)
      print('predictions shape: ', predictions.shape)
      print('prediction: ', predictions)
      print('targets: ', targets.shape)
      print('targets_len: ', target_lens)

      for i in range(len(target_lens)):
        print('target_lens[i]: ', target_lens[i])
        predictions[:, i, target_lens[i]:] = 0

      _, predictions = torch.max(predictions, dim = 2)
      predictions = predictions.float()
      print('predicted answer: ', predictions.shape)
      print('predicted answer: ', predictions)
      optimizer.zero_grad()
      loss = criterion(predictions,  targets)
      loss.backward() 
      optimizer.step()
      avg_loss += loss.item()
      if batch_index % 50 == 0:
          print("Time: ", time.time() - st)
          print("batch index: ",batch_index,' Loss: ', avg_loss / 50)
          avg_loss = 0
          st = time.time()
      batch_index += 1

      del features
      del features_lens
      del targets
      del target_lens
      del loss

def test(model, test_loader, epoch):
    ### Write your test code here! ###
    pass

## Define Model

In [30]:
model = Seq2Seq(input_dim=40, vocab_size=len(LETTER_LIST), hidden_dim=128)
optimizer = optim.Adam(model.parameters(), lr=0.001)
criterion = nn.CrossEntropyLoss(reduction=None)
nepochs = 25

for epoch in range(nepochs):
  train(model, train_loader, criterion, optimizer, epoch)
  # val()
  # test(model, test_loader, epoch)

key:  torch.Size([64, 128, 128])
value:  torch.Size([64, 128, 128])
text:  torch.Size([64, 191])
features shape:  torch.Size([64, 1018, 40])
predictions shape:  torch.Size([64, 191, 35])
prediction:  tensor([[[ 0.0159,  0.0031,  0.0410,  ...,  0.0112,  0.0001,  0.0411],
         [ 0.0283,  0.0083,  0.0462,  ...,  0.0218, -0.0032,  0.0445],
         [ 0.0307,  0.0038,  0.0467,  ...,  0.0170,  0.0022,  0.0510],
         ...,
         [-0.0061,  0.0074,  0.0379,  ...,  0.0057,  0.0041,  0.0407],
         [-0.0061,  0.0074,  0.0379,  ...,  0.0057,  0.0041,  0.0407],
         [-0.0061,  0.0074,  0.0379,  ...,  0.0057,  0.0041,  0.0407]],

        [[ 0.0191,  0.0085,  0.0404,  ..., -0.0004,  0.0185,  0.0435],
         [ 0.0236,  0.0177,  0.0334,  ...,  0.0013,  0.0214,  0.0381],
         [ 0.0178,  0.0191,  0.0207,  ..., -0.0094,  0.0173,  0.0306],
         ...,
         [-0.0062,  0.0071,  0.0378,  ...,  0.0058,  0.0037,  0.0403],
         [-0.0062,  0.0071,  0.0378,  ...,  0.0058,  0.0037,

ValueError: ignored